In [170]:
import pandas as pd

loc_data = pd.read_csv('source/cherryblossom_loc_selected_station.csv',
                     sep=',', engine='python')

## 시간별 지하철 역 입구 통행자 정보 가져오기

In [173]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()

# 수정
api_key = os.getenv("SKT_API_KEY")

## 역 코드로 출구 통행량 가져오기

In [174]:
def get_user_cnt_by_station_code(station_code, date):
    url = f"https://apis.openapi.sk.com/puzzle/subway/exit/raw/hourly/stations/{station_code}"

    headers = {
        "accept": "application/json",
        "Content-Type": "application/json",
        "appkey": api_key
    }

    params = {
        "gender": "all",
        "ageGrp": "all",
        "date": date
    }

    response = requests.get(url, headers=headers, params=params)
    
    return response

## 낮(09-16), 밤(17-23) 각각의 총 출구 통행량 구하기

In [192]:
def caculate_passenger_data(data, user_day_cnt_list, user_night_cnt_list):
    user_cnt_day = 0
    user_cnt_night = 0
    
    # 통행자 수 계산
    for raw in data['contents']['raw']:
        if raw['exit'] == str(row['exit_num']):
            if raw['datetime'] >= (str(date)+"090000") and raw['datetime'] < (str(date)+"170000"):
                user_cnt_day += (int(raw['userCount']) if raw['userCount'] != None else 0)
            elif raw['datetime'] >= (str(date)+"170000"):
                user_cnt_night += (int(raw['userCount']) if raw['userCount'] != None else 0)

    # 리스트에 저장
    user_cnt_day_list.append(user_cnt_day)
    user_cnt_night_list.append(user_cnt_night)

In [197]:
date = 20230401

for i in range(1, 8):
    user_cnt_day_list = []
    user_cnt_night_list = []
    
    for index, row in loc_data.iterrows():
        data = get_user_cnt_by_station_code(row['station_code'], date).json()
        caculate_passenger_data(data, user_day_cnt_list, user_night_cnt_list)
    # 통행량 데이터
    loc_data_temp = loc_data
    loc_data_temp['user_cnt_day'] = user_cnt_day_list
    loc_data_temp['user_cnt_night'] = user_cnt_night_list
    
    file_path = 'source/cherryblossom_final_{}.csv'.format(date)
    loc_data_temp.to_csv(file_path,
                      encoding='cp949', mode='w', index=False)
    date += 1